In [ ]:
import pandas as pd 
import re
from pprint import pprint
from tqdm import tqdm
import numpy as np

In [ ]:
PATH = '../../data/mirage_generic/'

In [ ]:
#df1 = pd.read_parquet(f'{PATH}dataset_100pkt_5f_payload_df_exact_noNullver_no0load_noMaps_b8829cf2.parquet')
#df = pd.read_parquet('dataset_100pkt_5f_payload_df_exact_noNullver_no0load_noMaps_noSniRem_b8829cf2.parquet')
#df = pd.read_parquet('dataset_100pkt_5f_payload_df_exact_noNullver_no0load_noMaps_noSniPadZero_b8829cf2.parquet')
df = pd.read_parquet('dataset_100pkt_5f_payload_df_exact_noNullver_no0load_noMaps_noSniObfZero_b8829cf2.parquet')

In [ ]:
df = df1

In [ ]:
l = "".join(map(chr,df['LOAD'][0][:500]))
l_nosni = "".join(map(chr,df_nosni['LOAD'][0][:500]))

In [ ]:
l_nosni

In [ ]:
l

In [ ]:
loads = df['LOAD']
client_hello_split = {
    'record_header' : 5,
    'handshake_header' : 4, 
    'client_ver' : 2, 
    'client_random' : 32, 
    'session_id_len' : 1,
    'session_id' : 0,
    'cypher_suit' : 0,
    'compression_method' : 0,
    'exts_len' : 2,
    'ext_type' : 2,
    'ext_len' : 2,
    'sni_len' : 2,
    'sni_type' : 1,
    'host_name_len' : 2,
    'host_name' : 0
}
TLS_RECORD_HEADERS = ['\x16\x03\x01', '\x16\x03\x02', '\x16\x03\x03', '\x16\x03\x04'] #TLS 1.0, 1.1, 1.2, 1.3
TLS_VERS = ['\x03\x01', '\x03\x02', '\x03\x03', '\x03\x04']

In [ ]:
def get_TLS_indices(load):
    indices_TLS = dict()
    l_string = "".join(map(chr, load))
    for record_header in TLS_RECORD_HEADERS:
            indices = [m.start() for m in re.finditer(record_header, l_string)]
            if indices != []:
                indices_TLS.setdefault(record_header, []).extend(indices)
    return indices_TLS

In [ ]:
def obfuscate_SNI(load, load_index, mode, pad_mode, index):
    client_hello_sect = dict()
    prev_byte = 0
    l_string = "".join(map(chr, load[load_index:]))
    
    if l_string[:2] != '\x16\x03':
        pprint(f'{l_string[:200]}')
        pprint(index)
        raise ValueError('Shift error')
    
    if l_string[5] != '\x01':
        return 'no_client_hello', None

    for field, byte_range in client_hello_split.items():
        if field == 'session_id': 
            # Skip session (len='session_id_len' byte)
            if len(client_hello_sect['session_id_len']) == 0:
                return 'session_len_0', None
            byte_range = ord(client_hello_sect['session_id_len']) 
            if byte_range == 0:
                # No session id is provided
                continue
        elif field == 'client_ver':
            # Check if the TLS version is encrypted
            if l_string[prev_byte : prev_byte + byte_range] not in TLS_VERS:
                return 'ver_mismatch', None
        elif field == 'cypher_suit': 
            # Skip cypher suit (len=2nd byte of 'cypher_suit')
            byte_range = ord(l_string[prev_byte + 1]) + 2 
        elif field == 'compression_method': 
            # Skip compression method (len=1st byte of 'compression_method')
            byte_range = ord(l_string[prev_byte]) + 1 
        elif field == 'ext_type':
            # Skip each extension that is not SNI_EXT (len=2nd byte of 'ext_len')
            i = 0
            while l_string[prev_byte : prev_byte + byte_range] != '\x00\x00':
                # The 3rd byte (before 'ext_len') in 'ext_type' should be 0
                if l_string[prev_byte + 2] != '\x00':
                    return 'ext_mismatch', None
                client_hello_sect[f'ext_{i}'] = l_string[prev_byte : prev_byte + 4]
                prev_byte += ord(l_string[prev_byte + 3]) + 4
                i += 1
        elif field == 'sni_type':
            # Only host name DNS 
            if l_string[prev_byte] != '\x00':
                return 'no_dns_hostname', None
        elif field == 'host_name': 
            # Get the host name given its size in the 2nd byte in 'host_name_len'
            byte_range = ord(client_hello_sect['host_name_len'][-1])
            
            # Set the pad type
            # Pad is random
            """ if pad_mode == 'rand':
                pad_len = 255 if mode == 'pad' else byte_range
                pad = np.random.choice(
                    np.split(load, [load_index + prev_byte - 7])[0],
                    size=(pad_len),
                    replace=True
                )
            # Pad is a list of zeros
            elif pad_mode == 'zero':
                pad_len = 255 if mode == 'pad' else byte_range
                pad = np.zeros(pad_len, dtype=np.int64)
            
            # Obfuscate the hostname
            if mode == 'obf':
                np.put(
                    load, 
                    list(range(load_index + prev_byte, load_index + prev_byte + byte_range)), 
                    pad
                )
            # Remove and pad the ext SNI
            elif mode == 'pad':
                splitted_load = np.split(
                    load, [load_index + prev_byte - 7, load_index + prev_byte + byte_range])
                load = np.concatenate([splitted_load[0], pad, splitted_load[2]]) 
            # Remove the ext SNI 
            elif mode == 'rem':
                splitted_load = np.split(
                    load, [load_index + prev_byte - 8, load_index + prev_byte + byte_range])
                load = np.concatenate([splitted_load[0], splitted_load[2]]) 
            else:
                raise ValueError('Invalid mode') """
            
        client_hello_sect[field] = l_string[prev_byte : prev_byte + byte_range]
        prev_byte += byte_range

    return client_hello_sect, load

In [ ]:
sni_to_indices = dict()
tls_exception_indices = dict()
oob_indices = []
no_tls_indices = []

for i, load in tqdm(loads.items(), total=len(loads)):
    load = load.copy()
    shift = 0
    indices_TLS_dict = get_TLS_indices(load)
    if not indices_TLS_dict:
        no_tls_indices.append(i)
        continue
    
    indices_TLS = []
    for v in indices_TLS_dict.values():
        indices_TLS.extend(v)
    indices_TLS.sort()

    for j, _ in enumerate(indices_TLS):
        
        #TODO calcolare lo shift
        current_TLS_dict = get_TLS_indices(load)
        current_TLS = []
        for v in current_TLS_dict.values():
            current_TLS.extend(v)
        current_TLS.sort()
        current_index = current_TLS[j]
    
        try:
            client_hello_sect, obf_load = obfuscate_SNI(
                load, current_index, mode='obf', pad_mode='zero', index=i)
            if obf_load is not None:
                df.at[i, 'LOAD'] = obf_load
                load = obf_load
        except IndexError as _:
            oob_indices.append(i)
            
        if isinstance(client_hello_sect, dict):
            sni_to_indices.setdefault(client_hello_sect['host_name'], []).append(i)
        elif isinstance(client_hello_sect, str):
            tls_exception_indices.setdefault(client_hello_sect, []).append(i)
        else:
            print('Error')

In [ ]:
df.to_parquet('dataset_100pkt_5f_payload_df_exact_noNullver_no0load_noMaps_noSniObfZero_b8829cf2.parquet')

In [ ]:
print(tls_exception_indices['ext_mismatch'])

In [ ]:
print(sni_to_indices.keys())

In [ ]:
print(sni_to_indices['api-d.dropbox.com'])

In [ ]:
l_string = "".join(map(chr, loads['192.168.20.105,58335,162.125.32.5,443,6_1495007218']))[0:300]
l_string

In [ ]:
index = '192.168.20.105,51390,162.125.66.5,443,6_1495010746'
load = loads[index]
load = load.copy()
client_hello_sect = None
shift = 0
indices_TLS_dict = get_TLS_indices(load)
indices_TLS = []
for v in indices_TLS_dict.values():
    indices_TLS.extend(v)
indices_TLS.sort()
print(indices_TLS)
print(len(load))

for i, load_index in enumerate(indices_TLS):
    
    current_TLS_dict = get_TLS_indices(load)
    current_TLS = []
    for v in current_TLS_dict.values():
        current_TLS.extend(v)
    current_TLS.sort()
    current_index = current_TLS[i]
    #print(load_index)
    try:
        """ print('shift')
        print(current_index)
        input() """
        client_hello_sect, obf_load = obfuscate_SNI(
            load, current_index, mode='pad', pad_mode='zero', index=index)
        if obf_load is not None:
            load = obf_load
    except IndexError as _:
        print('No match')
    pprint(client_hello_sect, sort_dicts=False)